In [1]:
# Install libraries within the notebook scope
sc.install_pypi_package("boto3")
sc.install_pypi_package("pandas")
sc.install_pypi_package("requests")
sc.install_pypi_package("s3fs")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1603063175185_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/2f/08/f1ff665147a5d75b871bbe5ba76916f6490419c52a33e588385c4b69281b/boto3-1.15.18-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2d/72/984ac8f33b5c8df5ff63f323a8724f65b4d0f8956968b942b77d35d3a1ef/botocore-1.18.18-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/69/79/e6afb3d8b0b4e96cefbdc690f741d7dd24547ff1f94240c997a26fa908d3/s3transfer-0.3.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9f/f0/a391d1463ebb1b233795cabfc0ef38d3db4442339de68f847026199e69d7/urllib3-1.25.10-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/25/47/22fc373440e144e2111363adaa07abb09ec1f03fbc071b6d9fc0bbf65f68/pandas-1.1.3-cp37-cp37m-manylinux1_x86_64.whl

  Using cached https://files.

In [2]:
import boto3
from datetime import datetime
from datetime import timedelta
import pandas as pd
from pyspark.sql.dataframe import DataFrame
from pyspark.sql import functions as f, types as t
from pathlib import Path
import requests
import s3fs
import subprocess
import timeit
from urllib.parse import urlparse

# Removes truncation of columns, column values in Pandas
# by default
pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)

# Monkey patching the DataFrame transform method for spark 2.4
def transform(self, f):
    return f(self)
DataFrame.transform = transform

# Override the timeit template to return the command's
# return value in addition to the time
# Reference: https://stackoverflow.com/questions/24812253/how-can-i-capture-return-value-with-python-timeit-module
timeit.template = """
def inner(_it, _timer{init}):
    {setup}
    _t0 = _timer()
    for _i in _it:
        retval = {stmt}
    _t1 = _timer()
    return _t1 - _t0, retval
"""

def shell_cmd(cmd):
    for line in subprocess.check_output(cmd, shell=True).split(b'\n'):
        print(line)

def timer_method(cmd):
    # Setting globals = globals() enables the timeit function
    # to return the value generated by cmd
    return timeit.timeit(cmd, number=1, globals = globals())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Set your s3 bucket name
This should be data-scale-oreilly-{your name}   
If you dont remember check the [S3 console](https://s3.console.aws.amazon.com/)

In [22]:
MY_BUCKET_NAME = "sev-data-scale-oreilly"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Lab 3.1 - Leveraging File Types

Write functions to:
* Write out the taxi-lookup dataset to local storage as csv, json, and parquet files
* Write a function to read the files and to print/return a dataframe containing the counts of zones per borough
* Time how long it takes to write out the different file types and to perform the aggregation using each file type

In [6]:
# These should point to the paths where the data from the taxi and taxi-lookup ingests were written
taxiPath = "hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json/"
taxiLookupPath = "hdfs:///tmp/data/nyc-taxi/zone-lookup/output/section2/json/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
def write_csv(inputDF):
    (inputDF
        .write
        .mode("overwrite")
        .option("header", True)
        .csv("hdfs:///tmp/data/nyc-taxi/taxi-zone/output/section3/csv"))

def write_json(inputDF):
    (inputDF
        .write
        .mode("overwrite")
        .json("hdfs:///tmp/data/nyc-taxi/taxi-zone/output/section3/json"))

def write_parquet(inputDF):
    (inputDF
        .write
        .mode("overwrite")
        .parquet("hdfs:///tmp/data/nyc-taxi/taxi-zone/output/section3/parquet"))
        
def read_test_csv():
    readDF = (spark
                 .read
                 .option('header', True).csv("hdfs:///tmp/data/nyc-taxi/taxi-zone/output/section3/csv"))
    readDF.show()
    dfCount = readDF.groupBy("borough").count()
    print(dfCount)
    return dfCount

def read_test_json():
    readDF = (spark
                 .read
                 .json("hdfs:///tmp/data/nyc-taxi/taxi-zone/output/section3/json"))
    readDF.show()
    dfCount = readDF.groupBy("borough").count()
    print(dfCount)
    return dfCount

def read_test_parquet():
    readDF = (spark
                 .read
                 .parquet("hdfs:///tmp/data/nyc-taxi/taxi-zone/output/section3/parquet"))
    readDF.show()
    dfCount = readDF.groupBy("borough").count()
    print(dfCount)
    return dfCount

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
inputDF = (spark
              .read
              .json(taxiLookupPath))
inputDF.printSchema()
inputDF.show()

print(f'CSV write time: {timer_method("write_csv(inputDF)")}')
print(f'JSON write time: {timer_method("write_json(inputDF)")}')
print(f'Parquet write time: {timer_method("write_parquet(inputDF)")}')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Borough: string (nullable = true)
 |-- LocationID: long (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)

+-------------+----------+--------------------+------------+
|      Borough|LocationID|                Zone|service_zone|
+-------------+----------+--------------------+------------+
|          EWR|         1|      Newark Airport|         EWR|
|       Queens|         2|         Jamaica Bay|   Boro Zone|
|        Bronx|         3|Allerton/Pelham G...|   Boro Zone|
|    Manhattan|         4|       Alphabet City| Yellow Zone|
|Staten Island|         5|       Arden Heights|   Boro Zone|
|Staten Island|         6|Arrochar/Fort Wad...|   Boro Zone|
|       Queens|         7|             Astoria|   Boro Zone|
|       Queens|         8|        Astoria Park|   Boro Zone|
|       Queens|         9|          Auburndale|   Boro Zone|
|       Queens|        10|        Baisley Park|   Boro Zone|
|     Brooklyn|        11|          Bath B

In [10]:
print(f'CSV read and transform time: {timer_method("read_test_csv()")}')
print(f'JSON read and transform time: {timer_method("read_test_json()")}')
print(f'Parquet read and transform time: {timer_method("read_test_parquet()")}')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+--------------------+------------+
|      Borough|LocationID|                Zone|service_zone|
+-------------+----------+--------------------+------------+
|          EWR|         1|      Newark Airport|         EWR|
|       Queens|         2|         Jamaica Bay|   Boro Zone|
|        Bronx|         3|Allerton/Pelham G...|   Boro Zone|
|    Manhattan|         4|       Alphabet City| Yellow Zone|
|Staten Island|         5|       Arden Heights|   Boro Zone|
|Staten Island|         6|Arrochar/Fort Wad...|   Boro Zone|
|       Queens|         7|             Astoria|   Boro Zone|
|       Queens|         8|        Astoria Park|   Boro Zone|
|       Queens|         9|          Auburndale|   Boro Zone|
|       Queens|        10|        Baisley Park|   Boro Zone|
|     Brooklyn|        11|          Bath Beach|   Boro Zone|
|    Manhattan|        12|        Battery Park| Yellow Zone|
|    Manhattan|        13|   Battery Park City| Yellow Zone|
|     Brooklyn|        1

### Lab 3.2 Partitioning

Write functions to:
* Based on the class discussion , update the following functions with the most approrpiate partitioning values
* Write a function to read the written files and to print/return a dataframe containing the counts of taxi rides per pickup_month
* Time how long it takes to write out the data using the different partitioning methodologies and to perform the aggregation using strategy

In [13]:
def write_paritioned_parquet(inputDF):
    (inputDF
        .write
        .mode("overwrite")
        .partitionBy('pickup_month')
        .parquet("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section3/partitioned"))
            
def write_coalesce_parquet(inputDF):
    (inputDF
        .coalesce(5)
        .write     
        .mode("overwrite")
        .partitionBy('pickup_month')
        .parquet("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section3/coalesced"))
            
def write_repartition_parquet(inputDF):
    (inputDF
        .repartition(5)
        .write
        .mode("overwrite")
        .partitionBy('pickup_month')
        .parquet("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section3/repartitioned"))
            
def write_sorted_parquet(inputDF): #RLE, need to verify that this still matters
    (inputDF.orderBy('pickup_month','passenger_count', 'PULocationID', 'DOLocationID', 'trip_distance', 'fare_amount', 'tip_amount', 'tpep_dropoff_datetime', 'tpep_pickup_datetime')
        .coalesce(5)     
        .write
        .mode("overwrite")
        .partitionBy('pickup_month')
        .parquet("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section3/sorted"))

# def write_parition(inputDF, partition):
#     writeDF = inputDF.filter(f"pickup_month == 'f{partition}'")
#     writeDF.show()
#     (writeDF.orderBy('passenger_count', 'PULocationID', 'DOLocationID', 'trip_distance', 'fare_amount', 'tip_amount', 'tpep_dropoff_datetime', 'tpep_pickup_datetime')
#         .repartition(5)
#         .write
#         .mode("overwrite")
#         .parquet(f"hdfs:///tmp/data/nyc-taxi/taxi-data/output/section3/manual/pickup_month={parition}"))

# def write_pre_defined_partitions(inputDF):
#     ##multiparll over list of partitions
#     #need to verify if this is actually faster
#     from multiprocessing import Process
#     partitionList = [x[0] for x in inputDF.select("pickup_month").distinct().collect()]
#     print(parititionList)
#     for partiion in partitionList:
#         writeProcess = Process(target=write_parition, args=(inputDF, partiion))
#         writeProcess.start()
#         writeProcess.join()
                
#     spark.sql("MSCK RPAIR TABLE table_name")
#     spark.sql("REFRESH TABLE table_name")

def read_test_parquet(readPath):
    readDF = (spark
                 .read
                 .parquet(readPath).sort("pickup_month", "passenger_count"))
    return readDF
def agg_test_parquet(baseDF):
    dfCount = baseDF.groupBy("pickup_month").count()
    print(dfCount)
    return dfCount

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Will talk through how to access the spark ui/view progress of the running code while this operation is running
inputDF = (spark
              .read
              .json(taxiPath)
              .withColumn("pickup_month", f.date_format("pickup_datetime", "yyyyMM")))
inputDF.show()

print(timer_method("write_paritioned_parquet(inputDF)"))
print(timer_method("write_coalesce_parquet(inputDF)"))
print(timer_method("write_repartition_parquet(inputDF)"))
print(timer_method("write_sorted_parquet(inputDF)"))
# print(timer_method("write_pre_defined_partitions(inputDF)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+----------+-------+--------+--------------------+--------------------+--------------------+-----+-----------+-----------------+---------------------+---------------------+--------------------+-----+-------+---------------+------------+--------------------+-------+------------------+----------+------------+------------+---------------------+--------------------+-------------+---------+----+------------+
|DOLocationID|PULocationID|RatecodeID|SR_Flag|VendorID|congestion_surcharge|dispatching_base_num|    dropoff_datetime|extra|fare_amount|hvfhs_license_num|improvement_surcharge|lpep_dropoff_datetime|lpep_pickup_datetime|month|mta_tax|passenger_count|payment_type|     pickup_datetime|service|store_and_fwd_flag|tip_amount|tolls_amount|total_amount|tpep_dropoff_datetime|tpep_pickup_datetime|trip_distance|trip_type|year|pickup_month|
+------------+------------+----------+-------+--------+--------------------+--------------------+--------------------+-----+----------

In [14]:
testPaths = [
    'hdfs:///tmp/data/nyc-taxi/taxi-data/output/section3/partitioned',
    'hdfs:///tmp/data/nyc-taxi/taxi-data/output/section3/coalesced',
    'hdfs:///tmp/data/nyc-taxi/taxi-data/output/section3/repartitioned',
    'hdfs:///tmp/data/nyc-taxi/taxi-data/output/section3/sorted',
#     'hdfs:///tmp/data/nyc-taxi/taxi-data/output/section3/manual/'
]

for testPath in testPaths:
    print(timer_method(f"read_test_parquet('{testPath}')"))
    baseDF = read_test_parquet(testPath)
    print(timer_method("agg_test_parquet(baseDF)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(0.1992224259993236, DataFrame[DOLocationID: bigint, PULocationID: bigint, RatecodeID: string, SR_Flag: string, VendorID: string, congestion_surcharge: string, dispatching_base_num: string, dropoff_datetime: string, extra: string, fare_amount: double, hvfhs_license_num: string, improvement_surcharge: string, lpep_dropoff_datetime: string, lpep_pickup_datetime: string, month: string, mta_tax: string, passenger_count: bigint, payment_type: string, pickup_datetime: string, service: string, store_and_fwd_flag: string, tip_amount: double, tolls_amount: string, total_amount: string, tpep_dropoff_datetime: string, tpep_pickup_datetime: string, trip_distance: double, trip_type: string, year: string, pickup_month: int])
DataFrame[pickup_month: int, count: bigint]
(0.014794507999795314, DataFrame[pickup_month: int, count: bigint])
(0.11366183999962232, DataFrame[DOLocationID: bigint, PULocationID: bigint, RatecodeID: string, SR_Flag: string, VendorID: string, congestion_surcharge: string, dispat

### Schema Management

In [15]:
# Normally want to leverage a respository
# In scala you could apply the types to the dataframe to create a dataset. That is not possible in PySpark 2.4
# Can leverage a SQL DDL or a spark structType. 
taxiSchema = inputDF.schema
print(spark.read.schema(taxiSchema).parquet('hdfs:///tmp/data/nyc-taxi/taxi-data/output/section3/sorted').count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

29381039

In [16]:
taxiSchema = t.StructType([ \
    t.StructField('DOLocationID',t.LongType(),True), \
    t.StructField('PULocationID',t.LongType(),True), \
    t.StructField('RatecodeID',t.StringType(),True), \
    t.StructField('SR_Flag',t.StringType(),True), \
    t.StructField('VendorID',t.StringType(),True), \
    t.StructField('congestion_surcharge',t.StringType(),True), \
    t.StructField('dispatching_base_num',t.StringType(),True), \
    t.StructField('dropoff_datetime',t.StringType(),True), \
    t.StructField('extra',t.StringType(),True), \
    t.StructField('fare_amount',t.DoubleType(),True), \
    t.StructField('hvfhs_license_num',t.StringType(),True), \
    t.StructField('improvement_surcharge',t.StringType(),True), \
    t.StructField('lpep_dropoff_datetime',t.StringType(),True), \
    t.StructField('lpep_pickup_datetime',t.StringType(),True), \
    t.StructField('mta_tax',t.StringType(),True), \
    t.StructField('passenger_count',t.LongType(),True), \
    t.StructField('payment_type',t.StringType(),True), \
    t.StructField('pickup_datetime',t.StringType(),True), \
    t.StructField('service',t.StringType(),True), \
    t.StructField('store_and_fwd_flag',t.StringType(),True), \
    t.StructField('tip_amount',t.DoubleType(),True), \
    t.StructField('tolls_amount',t.StringType(),True), \
    t.StructField('total_amount',t.StringType(),True), \
    t.StructField('tpep_dropoff_datetime',t.StringType(),True), \
    t.StructField('tpep_pickup_datetime',t.StringType(),True), \
    t.StructField('trip_distance',t.DoubleType(),True), \
#     StructField('trip_type',t.StringType(),True), \
    t.StructField('pickup_month',t.StringType(),True)])
print(spark.read.schema(taxiSchema).parquet('hdfs:///tmp/data/nyc-taxi/taxi-data/output/section3/sorted').count())

# stations = "col1 STRING, col2 INT"
spark.createDataFrame(inputDF.rdd,schema=taxiSchema,verifySchema=True).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o434.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 47.0 failed 4 times, most recent failure: Lost task 0.3 in stage 47.0 (TID 515, ip-172-31-4-197.us-east-2.compute.internal, executor 10): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/mnt/yarn/usercache/livy/appcache/application_1603063175185_0004/container_1603063175185_0004_01_000013/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/mnt/yarn/usercache/livy/appcache/application_1603063175185_0004/container_1603063175185_0004_01_000013/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/mnt/yarn/usercache/livy/appcache/application_1603063175185_0004/container_1603063175185_0004_01_000013/pyspark.zip/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/mnt/yarn/u

### Lab 3.3 - Case Study 3: Find the average taxi rides per zip code on the 10 worst air quality days of each month

Write functions to: 
* Select the best iteration of the taxi data for this workload
* Ingest the borough zipcode mapping and explode the zip code columns
* Ingest air quality data from: https://www.airnowapi.org/aq/observation/zipCode/historical/?format=application/json&zipCode={zipCode}&date={date}T00-0000&distance=100&API_KEY={apiKey}
    * https://docs.airnowapi.org/HistoricalObservationsByZip/docs
    * Make sure you have created a account at: https://docs.airnowapi.org/account/request/
    * Use the zip codes from the borough zipcode mapping
    * Due to API call limitations only pull 100 days of air quality for the following zipcodes (11212, 10023, 11374, 11414) starting on 2020/06/01
    * #### Be aware that you only have 500 requests per hour to that api endpoint
* Find the average taxi rides per borough on the 10 worst air quality days of each month
* Persist the resulting agg dataframe using .cache()
* Run and time the full ingest

Make sure to write the ingests with an eye on efficiency for this specific workload 

In [18]:
def get_taxi_df():
    taxiDF = spark.read.parquet("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section3/sorted")
#     taxi_lookup = spark.read.parquet("hdfs:///tmp/data/nyc-taxi/zone-lookup/output/section2/json/") #expected error
    taxi_lookup = spark.read.json("hdfs:///tmp/data/nyc-taxi/zone-lookup/output/section2/json/")
    taxi_filtered = (taxiDF
     .filter(taxiDF.pickup_datetime.isNotNull())
     .filter(taxiDF.dropoff_datetime.isNotNull()))
                     
    groupDF = taxi_filtered.join(taxi_lookup.select("Borough", "LocationID"), taxi_filtered.PULocationID == taxi_lookup.LocationID)
    groupDF.show()
    return groupDF

def get_zip_code_mapping_df():
    zipReadDF = spark.read.option('header', True).csv(f"s3://{MY_BUCKET_NAME}/data/borough-zip-mapping/ny-zip-codes.csv")
    returnZipDF = zipReadDF.select('Borough', 'Neighborhood', f.explode(f.split('ZIP Codes', ',')).alias('zip'))
    returnZipDF.show()
    return returnZipDF


def get_air_quality_df(zipDF):
#     zipList = [x[0] for x in zipDF.select('zip').collect()]
    zipList = ['11212', '10023']#, '11374', '11414']
    airQualitySchema = t.StructType([
        t.StructField("AQI", t.LongType(), True),
        t.StructField("Category", t.MapType(t.StringType(), t.LongType()), True),
        t.StructField("DateObserved", t.StringType(), True),
        t.StructField("HourObserved", t.LongType(), True),
        t.StructField("Latitude", t.DoubleType(), True),
        t.StructField("LocalTimeZone", t.StringType(), True),
        t.StructField("Longitude", t.DoubleType(), True),
        t.StructField("ParameterName", t.StringType(), True),
        t.StructField("ReportingArea", t.StringType(), True),
        t.StructField("StateCode", t.StringType(), True),
        t.StructField("zip", t.StringType(), True),
        ])
    
    airDF = spark.createDataFrame([], airQualitySchema)

    for zipCode in zipList:
        for d in range(0,99):
            startDate = datetime.strptime("2020-06-01", "%Y-%m-%d").date()
            dateDelta = timedelta(days=d)
            endDate = startDate + dateDelta
            
            apiPath = f"https://www.airnowapi.org/aq/observation/zipCode/historical/?format=application/json&zipCode={zipCode}&date={endDate}T00-0000&distance=100&API_KEY=8DFC7E6B-F641-41D9-95DC-9CF3B90AF038"
            request = requests.get(apiPath)
            requestDF = spark.createDataFrame(request.json())
            requestDF = requestDF.withColumn('zip', f.lit(zipCode))
            airDF = airDF.unionAll(requestDF)
        
    returnAirDF = airDF.withColumn("categoryNumber", f.col("Category.Number"))
    returnAirDF.show()
    
    return returnAirDF

def calculate_hottest_days(taxiDF, airQualityDF, zipDF):
    joinAirDF = airQualityDF.withColumn('air_day', f.date_format("dateObserved", "yyyyMMdd")).withColumnRenamed("zip", "air_zip")
    
    zipDF = zipDF.withColumn('zipBorough', f.col('Borough'))
    
    taxiZipDF = taxiDF.join(zipDF.select('zip', 'ZipBorough'), zipDF.zipBorough == taxiDF.Borough)
    
    joinTaxiDF = (taxiZipDF.withColumn("pickup_day", f.date_format("pickup_datetime", "yyyyMMdd"))
                        .withColumn("pickup_month", f.date_format("pickup_datetime", "yyyyMM")))

    joinCondition = [joinTaxiDF.pickup_day == joinAirDF.air_day, joinTaxiDF.zip == joinAirDF.air_zip]
    
    joinedAirDF = joinTaxiDF.join(joinAirDF.select('categoryNumber', 'air_day', 'air_zip'), joinCondition)
    
    aggDF = (joinedAirDF.groupBy('pickup_month', 'pickup_day', 'zip')
                        .agg(f.count('pickup_day').alias('count_rides'), f.avg('categoryNumber').alias('avg_cat')))
    
    win = Window.partitionBy("zip", "pickup_month").orderBy(f.desc("avg_cat", 'count_rides'))
    aggDF = aggDF.withColumn("row_num", f.row_number().over(win)).where("row_num >= 10")
    
    aggDF.show()
    return aggDF

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
def run_case_study():
    taxiDF = get_taxi_df()
    zipDF = get_zip_code_mapping_df()
    airQualityDF = get_air_quality_df(zipDF)
    airQualityDF.show()
    aggedDF = calculate_hottest_days(taxiDF, airQualityDF, zipDF)
    aggedDF.cache()
    print(aggedDF.count())
    # .cache is lazy evaluated, so we do the count to force the action

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
print(timer_method("run_case_study()"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'Window' is not defined
Traceback (most recent call last):
  File "<stdin>", line 44, in timer_method
  File "/usr/lib64/python3.7/timeit.py", line 233, in timeit
    return Timer(stmt, setup, timer, globals).timeit(number)
  File "/usr/lib64/python3.7/timeit.py", line 177, in timeit
    timing = self.inner(it, self.timer)
  File "<timeit-src>", line 6, in inner
  File "<stdin>", line 6, in run_case_study
  File "<stdin>", line 73, in calculate_hottest_days
NameError: name 'Window' is not defined



### Lab 3.4 - Write out data out to S3 for long term storage

Write functions to:
* Write the taxi/taxi-lookup data to S3 using the most appropriate storage format/partitioning methodology
* Write the cached dataframe from the previous usecase to S3 using the most appropriate storage format/partitioning methodology


In [24]:
# Normally want to avoid writing to EBS then copying to S3
# Intermediate workloads should be written to local EBS, while finished workloads/longterm storage should be read directly from/written directly to S3
# Luckily, EMRFS let's us write out directly to S3

def write_to_s3():
    taxiDF = (spark
              .read
              .json(taxiPath)
              .withColumn("pickup_month", f.date_format("pickup_datetime", "yyyyMM")))
    taxiLookupDF = (spark
              .read
              .json(taxiLookupPath)
              .withColumn("pickup_month", f.date_format("pickup_datetime", "yyyyMM")))
    
    (taxiDF.orderBy('pickup_month', 'passenger_count', 'PULocationID', 'DOLocationID', 'trip_distance', 'fare_amount', 'tip_amount', 'tpep_dropoff_datetime', 'tpep_pickup_datetime')
        .coalesce(5)     
        .write
        .mode("overwrite")
        .partitionBy('pickup_month')
        .parquet(f"s3://{MY_BUCKET_NAME}/tmp/data/nyc-taxi/taxi-data/final"))
    
    (taxiLookupDF.orderBy('borough', 'service_zone', 'locationID', 'zone')
        .coalesce(1)     
        .write
        .mode("overwrite")
        .parquet(f"s3://{MY_BUCKET_NAME}/tmp/data/nyc-taxi/taxi-lookup/final"))
    
    # We are leveraging (dangerously) a quirk of python here and a functionality of spark that let's us persist dataframes in memory. We are doing this to workaround a limitation of the timing function and the 500 api call limit from the air quality api
    (aggedDF.orderBy('pickup_month', 'row_num', 'zip', 'avg_cat', 'count_rides')
        .coalesce(1)     
        .write
        .mode("overwrite")
        .partitionBy('pickup_month')
        .parquet(f"s3://{MY_BUCKET_NAME}/tmp/data/nyc-taxi/taxi-agg/final"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
write_to_s3()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"cannot resolve '`pickup_datetime`' given input columns: [Borough, LocationID, Zone, service_zone];;\n'Project [Borough#14408, LocationID#14409L, Zone#14410, service_zone#14411, date_format('pickup_datetime, yyyyMM, Some(UTC)) AS pickup_month#14416]\n+- Relation[Borough#14408,LocationID#14409L,Zone#14410,service_zone#14411] json\n"
Traceback (most recent call last):
  File "<stdin>", line 13, in write_to_s3
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 2000, in withColumn
    return DataFrame(self._jdf.withColumn(colName, col._jc), self.sql_ctx)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: "cannot resolve '`pickup_datetime`' given input columns: [Borough, Locat